In [167]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import random
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
# from sklearn.cluster import OPTICS
import collections

In [168]:
# Remove the outliers
def remove_outliers(file_path):
    # return the list of dataframes
    dataframe_lists = []
    # Creating the dataframe and selecting the required columns
    for i in range(64):
        df = pd.read_csv(file_path, usecols=[1,2,3,4], skiprows=i*72000, nrows = 72000, names=["lz","X","Y","Z"])
        df['radiusSquare'] = df['X']*df['X']+df['Y']*df['Y']+df['Z']*df['Z']
        df['radius'] = np.sqrt(df['radiusSquare']).round(1)
        df['freq'] = df.groupby(['lz','radius'])['radius'].transform('count')
        for j in range(64):
            maxfreq = df[(df['lz']==j) & (df['radius']!=0)]['freq'].max()
            while maxfreq>100:
                df.drop(df[(df['lz']==j) & (df['freq']==maxfreq)].index, inplace=True)
                maxfreq = df[(df['lz']==j) & (df['radius']!=0)]['freq'].max()
                df.drop(df[(df['lz']==j) & (df['radius']==0)].index, inplace=True)
        dataframe_lists.append(df.iloc[:,1:4])
    return dataframe_lists

In [169]:
# atm = remove_outliers("/home/kia/Collected-Datasets/DEBS2019/debs2019_initial_dataset/Atm/in.csv")

# This the second data set that contains multiple objects
allData= remove_outliers("/home/kia/Collected-Datasets/DEBS2019/debs2019_dataset2/in.csv")


mX=np.array(allData[0]['X'])
mY=np.array(allData[0]['Y'])
mZ=np.array(allData[0]['Z'])




In [170]:
DataForClustering =  np.array(list(zip(mX, mY, mZ))) 

clustering = DBSCAN(eps=1, min_samples=10).fit(DataForClustering)

labels = clustering.labels_


labels = labels + 1 
print(max(labels))
print(min(labels))

38
0


In [171]:
def isInt(value):
    try:
        int(value)
        return True
    except:
        return False


numOfObjects = []

# Out put data.
# We need to get the number of objects. 
pathOutput="/home/kia/Collected-Datasets/DEBS2019/debs2019_dataset2/out.csv"

with open(pathOutput) as fp:  
   line = fp.readline()
   cnt = 1
   while line:
    # print("Line {}: {}".format(cnt, line.strip()))
        lineArr=line.split(",")
        num = 0
        for index in range(1, len(lineArr)):
            if(isInt(lineArr[index])):
               num +=int(lineArr[index])
        numOfObjects.append(num)
        #print(line.split(",")[1])
        line = fp.readline()
        cnt += 1
        
        
        
print(max(numOfObjects))

aList=np.array(numOfObjects)
print(np.mean(aList))
print(np.median(aList))

50
29.186
29.0


In [172]:
# clustering with DBSCAN 
def doClusteringWithDBSCAN(data, eps=1, min_samples=16):
    
    clustering = DBSCAN(eps, min_samples).fit(data)

    labels = clustering.labels_

    labels = labels +1 
    
    return max(labels)

In [173]:
from scipy import stats 


In [174]:
# run a test. 
import math 

difs=[]
dens=[]

sum = 0 
for i in range(64):
    data =  np.array(list(zip(np.array(allData[i]['X']), np.array(allData[i]['Y']), np.array(allData[i]['Z'])))) 
    
    #gaussian_kernel = stats.gaussian_kde(np.array(allData[i]['X']))
    # values = np.argmax(gaussian_kernel)
    # print(gaussian_kernel)
    a= np.square (np.array(allData[i]['X'])) + np.array(allData[i]['Y']) + np.square (np.array(allData[i]['Z'])) 
    # a= np.array(allData[i]['Y'])

    # hist, bin_edges = np.histogram(a, density=True)
    # print(hist)
    # print(i, max(hist))
    # min_samples_value=round(0.0105*(1/max(hist)), 0)
    
    # min_samples_value=round(-2.2 * math.log(max(hist)), 0)
    
    # min_samples_value= 80*max(hist)
    min_samples_value=16
    eps_value=1
    print(i, max(a), round(eps_value, 2) , round(min_samples_value, 2))
    

    numberOfClusters = doClusteringWithDBSCAN(data, eps=eps_value, min_samples=min_samples_value)
    print(i, " Number Of Clusters: ", numberOfClusters, ", Difference is: ", numOfObjects[i] - numberOfClusters)
    sum = sum + (abs(numOfObjects[i] - numberOfClusters))
    
    dens.append(max(a))
    difs.append(numOfObjects[i] - numberOfClusters)
    
    
print(sum)

# print(dens)
# print(difs)

print(np.corrcoef(dens, difs))

0 14362.107467897948 1 16
0  Number Of Clusters:  32 , Difference is:  1
1 14360.539251224742 1 16
1  Number Of Clusters:  26 , Difference is:  -1
2 14354.502273646314 1 16
2  Number Of Clusters:  28 , Difference is:  -3
3 14320.71906901381 1 16
3  Number Of Clusters:  17 , Difference is:  0
4 14358.634878175537 1 16
4  Number Of Clusters:  20 , Difference is:  -4
5 14358.89280100912 1 16
5  Number Of Clusters:  35 , Difference is:  -1
6 14358.783788932851 1 16
6  Number Of Clusters:  33 , Difference is:  2
7 14351.874143458026 1 16
7  Number Of Clusters:  24 , Difference is:  -4
8 14350.613622470155 1 16
8  Number Of Clusters:  16 , Difference is:  1
9 14360.516636177545 1 16
9  Number Of Clusters:  33 , Difference is:  -1
10 14361.475928564338 1 16
10  Number Of Clusters:  41 , Difference is:  -2
11 14360.718481942182 1 16
11  Number Of Clusters:  43 , Difference is:  4
12 14360.85913381866 1 16
12  Number Of Clusters:  42 , Difference is:  -2
13 14356.823248703728 1 16
13  Number Of

In [47]:
i=52

data =  np.array(list(zip(np.array(allData[i]['X']), np.array(allData[i]['Y']), np.array(allData[i]['Z'])))) 
a=np.array(allData[i]['Z'])

hist, bin_edges = np.histogram(a, density=True)

max(hist)
#plt.hist(a, bins='auto')  # arguments are passed to np.histogram
#plt.title("Histogram with 'auto' bins")
#plt.show()
#numberOfClusters = doClusteringWithDBSCAN(data, eps=1, min_samples=9)
#print(i, " Number Of Clusters: ", numberOfClusters, ", Difference is: ", numOfObjects[i] - numberOfClusters)

0.022130927154838345